In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/openintro-possum/possum.csv")
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

In [ ]:
def prepro(df):
    
    df = df.copy()
    
    for col in df.columns:
        if df[col].dtype != "object":
            df[col].fillna(df[col].mean(),inplace=True)
            
    df.drop(columns=["case"])
    
    df["Pop"] = df["Pop"].replace({"Vic":0,"other":1})
    
    X = df.drop("sex",axis=1)
    y = df["sex"]

    X = pd.DataFrame(minmax_scale(X),index=X.index,columns=X.columns)
    
    return train_test_split(X, y, test_size=0.3)
    

In [ ]:
X_train,X_test,y_train,y_test = prepro(df)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
knn = KNeighborsClassifier()
logmod = LogisticRegression()
svc = SVC()

knn.fit(X_train,y_train)
logmod.fit(X_train,y_train)
svc.fit(X_train,y_train)

print("K Nearest Neighbours Model Score: {}".format(knn.score(X_test,y_test)))
print("")
print("Logistic Regression Model Score: {}".format(logmod.score(X_test,y_test)))
print("")
print("Support Vector Model Score: {}".format(svc.score(X_test,y_test)))

In [ ]:
errors = []
for i in range(1,30):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    
    p = knn.predict(X_test)
    
    errors.append(np.mean(p != y_test))

plt.figure(figsize=(12,6))
plt.plot(range(1,30),errors,ls="--",marker="X",
        markerfacecolor="red")

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train,y_train)
print("KNN Score: "+str(knn.score(X_test,y_test)))

In [ ]:
predicted = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))

Can principal component analysis help imporve accuracy?

In [ ]:
df.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def prepro(df):
    
    df = df.copy()
    
    for col in df.columns:
        if df[col].dtype != "object":
            df[col].fillna(df[col].mean(),inplace=True)
            
    
    df["Pop"] = df["Pop"].replace({"Vic":0,"other":1})
    
    X = df.drop(columns=["sex","case"],axis=1)
    y = df["sex"].replace({"m":1,"f":0})
    
    pca = PCA(n_components=2)
    X = pca.fit_transform(X)
    
    return X,y

In [ ]:
X,y = prepro(df)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X[:,0],X[:,1],c=y)

unfortunately there is no clear definition between our two classes even when reduced to two principal components

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
knn = KNeighborsClassifier()
logmod = LogisticRegression()
svc = SVC()

knn.fit(X_train,y_train)
logmod.fit(X_train,y_train)
svc.fit(X_train,y_train)

print("K Nearest Neighbours Model Score: {}".format(knn.score(X_test,y_test)))
print("")
print("Logistic Regression Model Score: {}".format(logmod.score(X_test,y_test)))
print("")
print("Support Vector Model Score: {}".format(svc.score(X_test,y_test)))

In [ ]:
preds = logmod.predict(X_test)

print(classification_report(y_test,preds))

In [ ]:
def prepro(df):
    
    df = df.copy()
    
    for col in df.columns:
        if df[col].dtype != "object":
            df[col].fillna(df[col].mean(),inplace=True)
            
    
    df["Pop"] = df["Pop"].replace({"Vic":0,"other":1})
    
    X = df.drop(columns=["sex","case"],axis=1)
    y = df["sex"].replace({"m":1,"f":0})
    
    pca = PCA(n_components=3)
    X = pca.fit_transform(X)
    
    return X,y

In [ ]:
X,y = prepro(df)

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
pcdf = pd.DataFrame(X,columns=["PC1","PC2","PC3"])

In [ ]:
pcdf = pd.concat([pcdf,pd.DataFrame(y)],axis=1)
pcdf.head()

In [ ]:
pcdf["sex_as_str"] = pcdf["sex"].replace({1:"m",0:"f"})

In [ ]:
pcdf.iplot(kind="scatter3d",
          x="PC1",y="PC2",z="PC3",
          categories="sex_as_str",
          size=4)

Still, with a 3d plot of three principal components, it is difficult to see a delineation between classes.
Nevertheless, I will run the logistic model on the three principal components

In [ ]:
X = pcdf.drop(["sex_as_str","sex"],axis=1)

y = pcdf["sex"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

logmod = LogisticRegression()

logmod.fit(X_train,y_train)

logmod.score(X_test,y_test)

In [ ]:
p = logmod.predict(X_test)

In [ ]:
print(classification_report(y_test,p))